# Setting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Stage2/code') 

In [3]:
os.getcwd()

'/content/drive/MyDrive/Stage2/code'

라이브러리 다운로드

In [4]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 46.9MB 92kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 10.7MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595680 sha256=a8fd9bddd2788a3d6123dbe97feac473681ab99e920c4d9f1daab9cbc01fed46
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 13.2MB/s 
     |████████████████████████████████| 757kB 14.7MB/s 
     |████████████████████████████████| 901kB 31.6MB/s 
     |████████████████████████████████| 3.0MB 48.5MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-6vmsfo_v
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-6vmsfo_v
  Created wheel for kobe

In [5]:
!pip install pororo

     |████████████████████████████████| 266kB 16.1MB/s 
     |████████████████████████████████| 471kB 25.3MB/s 
     |████████████████████████████████| 748.8MB 18kB/s 
     |████████████████████████████████| 1.5MB 29.5MB/s 
     |████████████████████████████████| 5.9MB 12.2MB/s 
     |████████████████████████████████| 276kB 53.8MB/s 
     |████████████████████████████████| 3.1MB 38.3MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 1.7MB 36.5MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 2.3MB 41.5MB/s 
     |████████████████████████████████| 184kB 54.3MB/s 
     |████████████████████████████████| 133kB 51.9MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 3.3MB 41.2MB/s 
     |████████████████████████████████| 112kB 52.4MB/s 
     |████████████████████████████████| 645kB 47.4MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none

라이브러리 불러오기

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import pandas as pd
import numpy as np
import re
import tarfile
import pickle as pickle
from tqdm import tqdm
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.model_selection import train_test_split,StratifiedKFold

from transformers import *
from tqdm import tqdm

In [8]:
from pororo import Pororo

GPU 설정

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")

In [10]:
device

device(type='cuda', index=0)

# Preprocessing

In [24]:
data_path = "/content/drive/MyDrive/Stage2/input/data/"

In [25]:
dataset = pd.read_csv(data_path+'train/train.tsv', delimiter='\t', header=None)

In [26]:
dataset

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
...,...,...,...,...,...,...,...,...,...
8995,wikipedia-5414-12-15-21-0-4,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,15,21,2002년,0,4,관계_없음
8996,wikipedia-10384-4-12-14-0-1,일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...,토요타,12,14,일본,0,1,단체:본사_국가
8997,wikipedia-25913-6-8-10-93-106,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...,방덕룡,8,10,선무원종공신(宣武原從功臣),93,106,인물:직업/직함
8998,wikitree-12062-15-0-3-46-47,LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...,LG전자,0,3,북미,46,47,관계_없음


In [27]:
test_dataset = pd.read_csv(data_path+'test/test.tsv', delimiter='\t', header=None)
test_dataset

,0,1,2,3,4,5,6,7,8
0,wikipedia-15133-19-0-2-45-46,"이은주의 첫 영화로, 그녀가 강수연(정화 역)의 동생 세화 역으로 출연한 박종원 감...",이은주,0,2,감독,45,46,blind
1,wikitree-10241-0-0-4-9-11,경성대학교(총장 송수건) 미술학과 졸업예정 학생들이 부산 지역의 유망한 청년예술가에...,경성대학교,0,4,송수건,9,11,blind
2,wikipedia-24909-38-83-89-45-48,하더라도 만일 그녀의 아들에게 계승시킬 수 있다면 영국 왕 에드워드 2세의 왕비 이...,에드워드 3세,83,89,이사벨라,45,48,blind
3,wikitree-29775-2-38-43-17-18,정성으로 담근 700포기의 김장김치는 발굴된 저소득 가정 55세대에 새마을부녀회 회...,새마을부녀회,38,43,김치,17,18,blind
4,wikipedia-20874-21-8-9-30-32,"노트북은 이들 정부에 판매될 것이며 ""하나의 노트북을 한 명의 어린이마다""라는 정책...",정부,8,9,한 명,30,32,blind
...,...,...,...,...,...,...,...,...,...
995,wikipedia-5911-11-14-18-21-23,"1940년 7월 10일, 나치 독일의 프랑스 침공으로 국회는 페텡에게 권력을 위임했다.",나치 독일,14,18,프랑스,21,23,blind
996,wikipedia-17379-20-200-205-179-181,(하지만 Vasco는 '스핏파이어(spit fire)(spit fire)'란 프로젝...,DJ DOC,200,205,박정환,179,181,blind
997,wikitree-59796-5-32-35-37-39,"또한 (사)한국여성농업인 광양시 연합회(도미화 회장)는 ㈜광양교통(이순심 대표)에,...",광양교통,32,35,이순심,37,39,blind
998,wikipedia-1829-41-9-10-0-1,마코가 항의하자 벵어는 마코를 배신자로 부르며 학생들에게 마코를 처형대로 끌어내도록...,벵어,9,10,마코,0,1,blind


### Pororo 의 Ner을 이용해 entity 추출해 sentence entity 양옆에 추가해주기

In [28]:
from collections import Counter

In [29]:
def add_entity_tokens(sentence, a1, a2, b1, b2):
    new_sentence = None
    ner = Pororo(task="ner", lang="ko")
    
    if a1 > b1:
        e1, e2 = sentence[b1:b2+1], sentence[a1:a2+1]
        n1, n2 = Counter([e[1] for e in ner(e1)]).most_common(1)[0][0], Counter([e[1] for e in ner(e2)]).most_common(1)[0][0]
        ner1, ner2 = "∮" + n1 + "∮", "∑" + n2 + "∑"
        new_sentence = sentence[:b1] + "∀" + ner1 + sentence[b1:b2+1] + "∀" + sentence[b2+1:a1] + "∏" + ner2 + sentence[a1:a2+1] + "∏" + sentence[a2+1:]
    else:
        e1, e2 = sentence[a1:a2+1], sentence[b1:b2+1]
        n1, n2 = Counter([e[1] for e in ner(e1)]).most_common(1)[0][0], Counter([e[1] for e in ner(e2)]).most_common(1)[0][0]
        ner1, ner2 = "∑" + n1 + "∑", "∮" + n2 + "∮"
        new_sentence = sentence[:a1] + "∏" + ner1 + sentence[a1:a2+1] + "∏" + sentence[a2+1:b1] + "∀" + ner2 + sentence[b1:b2+1] + "∀" + sentence[b2+1:]
    return new_sentence

In [30]:
lst = []
for i in tqdm(range(len(dataset))):
    now_data = dataset.iloc[i]
    new_line = add_entity_tokens(now_data[1],now_data[3],now_data[4],now_data[6],now_data[7])
    lst.append(new_line)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
100%|██████████| 9000/9000 [7:54:15<00:00,  3.16s/it]


In [32]:
dataset[1] = lst
dataset

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 ∀∮ARTIFACT∮자동차∀의 브랜드로는 ∏∑OR...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,선거에서 ∏∑ORGANIZATION∑민주당∏은 해산 전 의석인 230석에 한참 못 ...,민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,∏∑ORGANIZATION∑유럽 축구 연맹∏(∀∮ORGANIZATION∮UEFA∀)...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,용병 ∀∮CIVILIZATION∮공격수∀ 챠디의 부진과 시즌 초 활약한 ∏∑PERS...,강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,∏∑LOCATION∑람캄행∏ 왕은 1237년에서 1247년 사이 수코타이의 왕 ∀∮...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
...,...,...,...,...,...,...,...,...,...
8995,wikipedia-5414-12-15-21-0-4,∀∮DATE∮2002년∀ FIFA 월드컵 ∏∑COUNTRY∑사우디아라비아∏와의 1차...,사우디아라비아,15,21,2002년,0,4,관계_없음
8996,wikipedia-10384-4-12-14-0-1,∀∮COUNTRY∮일본∀의 2대 메이커인 ∏∑ORGANIZATION∑토요타∏와 닛산...,토요타,12,14,일본,0,1,단체:본사_국가
8997,wikipedia-25913-6-8-10-93-106,방호의의 손자 ∏∑PERSON∑방덕룡∏(方德龍)은 1588년(선조 21년) 무과에 ...,방덕룡,8,10,선무원종공신(宣武原從功臣),93,106,인물:직업/직함
8998,wikitree-12062-15-0-3-46-47,∏∑ORGANIZATION∑LG전자∏는 올해 초 국내시장에 출시한 2020년형 ‘L...,LG전자,0,3,북미,46,47,관계_없음


In [33]:
dataset.to_csv(data_path+'/train/new_train.tsv', sep='\t', index=False, header=False)

In [ ]:
lst = []
for i in tqdm(range(len(test_dataset))):
    now_data = test_dataset.iloc[i]
    new_line = add_entity_tokens(now_data[1],now_data[3],now_data[4],now_data[6],now_data[7])
    lst.append(new_line)

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 1000/1000 [51:09<00:00,  3.07s/it]


In [ ]:
test_dataset[1] = lst
test_dataset.to_csv(data_path+'/test/new_test.tsv', sep='\t', index=False, header=False)

In [34]:
t = pd.read_csv(data_path+'train/new_train.tsv', delimiter='\t', header=None)
t

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 ∀∮ARTIFACT∮자동차∀의 브랜드로는 ∏∑OR...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,선거에서 ∏∑ORGANIZATION∑민주당∏은 해산 전 의석인 230석에 한참 못 ...,민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,∏∑ORGANIZATION∑유럽 축구 연맹∏(∀∮ORGANIZATION∮UEFA∀)...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,용병 ∀∮CIVILIZATION∮공격수∀ 챠디의 부진과 시즌 초 활약한 ∏∑PERS...,강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,∏∑LOCATION∑람캄행∏ 왕은 1237년에서 1247년 사이 수코타이의 왕 ∀∮...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
...,...,...,...,...,...,...,...,...,...
8995,wikipedia-5414-12-15-21-0-4,∀∮DATE∮2002년∀ FIFA 월드컵 ∏∑COUNTRY∑사우디아라비아∏와의 1차...,사우디아라비아,15,21,2002년,0,4,관계_없음
8996,wikipedia-10384-4-12-14-0-1,∀∮COUNTRY∮일본∀의 2대 메이커인 ∏∑ORGANIZATION∑토요타∏와 닛산...,토요타,12,14,일본,0,1,단체:본사_국가
8997,wikipedia-25913-6-8-10-93-106,방호의의 손자 ∏∑PERSON∑방덕룡∏(方德龍)은 1588년(선조 21년) 무과에 ...,방덕룡,8,10,선무원종공신(宣武原從功臣),93,106,인물:직업/직함
8998,wikitree-12062-15-0-3-46-47,∏∑ORGANIZATION∑LG전자∏는 올해 초 국내시장에 출시한 2020년형 ‘L...,LG전자,0,3,북미,46,47,관계_없음
